In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from utils import *

In [3]:
def filelist(root, file_type):
    """Returns a fully-qualified list of filenames under root directory"""
    return [os.path.join(directory_path, f) for directory_path, directory_name, 
            files in os.walk(root) for f in files if f.endswith(file_type)]

In [4]:
def generate_df (anno_path, images_path):
    annotations = filelist(anno_path, '.xml')
    anno_list = []
    for anno_path in annotations:
        root = ET.parse(anno_path).getroot()
        anno = {}
        anno['filename'] = Path(str(images_path) + '/'+ root.find("./filename").text)
        anno['width'] = root.find("./size/width").text
        anno['height'] = root.find("./size/height").text
        if root.find("./object/name")!= None:
            anno['class'] = root.find("./object/name").text
        else:
            anno['class'] = np.NaN
        
        bboxes = []
        for neighbor in root.iter('bndbox'):
            if root.find("./object/bndbox/xmin") != None:
                xmin = int(float(neighbor.find('xmin').text))
                ymin = int(float(neighbor.find('ymin').text))
                xmax = int(float(neighbor.find('xmax').text))
                ymax = int(float(neighbor.find('ymax').text))

                bboxes.append([xmin, ymin, xmax, ymax])
                anno['bboxes'] = bboxes
            else: 
                anno['bboxes'] = np.NaN     
       
        anno_list.append(anno)
    return pd.DataFrame(anno_list)

In [5]:
root_img = '/mnt/hdd/astamoulakatos/NMIS/images/'
root_annotation = '/mnt/hdd/astamoulakatos/NMIS/bolts-xml-annotations/Annotations/'
df = generate_df(root_annotation, root_img)

In [6]:
df.head()

,filename,width,height,class,bboxes
0,/mnt/hdd/astamoulakatos/NMIS/images/2021-10-29...,4032,3024,Bolt,"[[3530, 2445, 3730, 2644]]"
1,/mnt/hdd/astamoulakatos/NMIS/images/2021-10-29...,4032,3024,Bolt,"[[351, 319, 546, 519]]"
2,/mnt/hdd/astamoulakatos/NMIS/images/2021-10-29...,4032,3024,Bolt,"[[287, 1023, 486, 1227], [1404, 2490, 1604, 26..."
3,/mnt/hdd/astamoulakatos/NMIS/images/2021-10-29...,4032,3024,Bolt,"[[272, 1018, 466, 1212], [1399, 2495, 1599, 26..."
4,/mnt/hdd/astamoulakatos/NMIS/images/2021-10-29...,4032,3024,Bolt,"[[356, 389, 561, 588]]"


In [7]:
df = df.dropna()

In [8]:
df = df.reset_index(drop=True)

In [9]:
len(df)

107

In [10]:
def bolts_count(x):
    return int(str(x).split('_')[1])

In [17]:
df['total_bolts'] = df.filename.apply(bolts_count)

In [18]:
df.head()

,filename,width,height,class,bboxes,total_bolts
0,/mnt/hdd/astamoulakatos/NMIS/images/2021-10-29...,4032,3024,Bolt,"[[3530, 2445, 3730, 2644]]",1
1,/mnt/hdd/astamoulakatos/NMIS/images/2021-10-29...,4032,3024,Bolt,"[[351, 319, 546, 519]]",1
2,/mnt/hdd/astamoulakatos/NMIS/images/2021-10-29...,4032,3024,Bolt,"[[287, 1023, 486, 1227], [1404, 2490, 1604, 26...",3
3,/mnt/hdd/astamoulakatos/NMIS/images/2021-10-29...,4032,3024,Bolt,"[[272, 1018, 466, 1212], [1399, 2495, 1599, 26...",3
4,/mnt/hdd/astamoulakatos/NMIS/images/2021-10-29...,4032,3024,Bolt,"[[356, 389, 561, 588]]",1


In [14]:
df.to_csv('dataset.csv', index=False)